In [ ]:
#Install EMIST library, import datasets of letters, Matplotlib
#!pip install emnist
from emnist import list_datasets
import pandas as pd
import pyspark
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sc

In [ ]:
# stop previus context
sc.stop()

In [ ]:
spark.stop()

spark = pyspark.SparkContext(appName="Save EMNIST to HDFS")


In [ ]:
spark

loading the dataset

In [ ]:
#Download dataset and it is 536 MB
list_datasets()

In [ ]:
#Download images of letters from training samples or test samples
#from emnist import extract_test_samples
from emnist import extract_training_samples
images, labels = extract_training_samples('letters')


In [ ]:
images.shape, labels.shape


# Saving on Hadoop via Pyspark
    to save the dataset into Hadoop,  first it must be loaded into a RDD (Resilient Distributed Dataset) 
    It's a data structure in the Apache Spark framework designed for large-scale data processing on clusters.

In [ ]:
#creating rdd files
# As spark works in partition, the number of partitions was increased to distribute the workload more evenly across tasks

images_rdd = spark.parallelize(images)
#images_rdd = images_rdd.repartition(12) 

labels_rdd = spark.parallelize(labels)
#labels_rdd = images_rdd.repartition(12)

In [ ]:
# saving on hadoop file system
hdfs_path = "hdfs://192.168.1.23:9000/emnist/"
# Save RDDs as parquet files in the specified HDFS path
images_rdd.saveAsTextFile(hdfs_path+"images")
labels_rdd.saveAsTextFile(hdfs_path+"labels")



# Loading RDD From hadoop
    for handling large size data, Libraries like dask and modin should be used,  because they are designed to handle large datasets efficiently on a single machine or distributed cluster. 

In [ ]:
hdfs_path = "hdfs://localhost:9000/emnist/"

# Load data from HDFS using dask (replace 'path/to/data' with the actual location)
images_rdd = spark.textFile(hdfs_path+"images")
labels_rdd = spark.textFile(hdfs_path+"labels")




# Plot sample of images

In [ ]:
# Split the RDD into 95% and 5% portions (adjust ratios as needed)
training_data, sample_data = images_rdd.randomSplit([0.95, 0.05])

# Collect the sample data
sample_images = sample_data.collect()

In [ ]:
fig,axes = plt.subplots(3,5,figsize=(10,8))
for i,ax in enumerate(axes.flat):
    ax.imshow(sample_images[i].reshape([28,28]) )


# Preprocesssing
## 1. Normalisation and reshaping of data
    Before feeding in the data to the model we will normalise and reshape the data given to us. This will decrease the complexity of the models and make the model work efficiently as less complex numbers will be there to process.

one-hot encoding using keras' numpy-related utilities


Adding Callback API's to save best weights and change lr¶


In [ ]:

model = Sequential()

# convolutional layer   - 
# ConvD(25 neurons)
#kernel_size 3,3 = filter size =  3xx
# strides =  how many square move by time  =   1 by on
# padding = Valid
# Activation function = relu  -  check ann to understand each activation function
# input_shate =  size of 3d picture (28x28x1

model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))


# 
model.add(MaxPool2D(pool_size=(1,1)))

# flatten output of conv
model.add(Flatten())



# hidden layer
# Deense with 100 neurons   -activation relu -  check why not sigmoid
model.add(Dense(100, activation='relu'))




# output layer
#37 - the numbers from 0 to 9  and letters A-Z,  and actication softmax
model.add(Dense(37, activation='softmax'))

compiling the sequential model

In [ ]:

# Its a multiple categorical ,  so we user categorical_crossentropy as  less function.
# optimizaer adam

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

training the model for 10 epochs

In [ ]:
# Batch_size = get 128 data with and pass for the model 10 times (epochs)

model.fit(X_train, y_train, batch_size=128, epochs=15, validation_data=(X_test, y_test))

Save the model to disk.

In [ ]:
model.save_weights('cnn_B.h5')

Load the model from disk using later:

In [ ]:
model.load_weights('cnn_B.h5')

Predict on the first 5 test images.

In [ ]:
predictions = model.predict(X_test [:10]) 


Print our model's predictions.

In [ ]:
print(np.argmax(predictions, axis=1))

Check our predictions against the ground truths.

In [ ]:
print(y_test[:10]) 